In [ ]:
# Cell 1 — Objective: Install only the required packages for this notebook; import nothing sensitive here.
# Unpinned: installs latest oci, ipywidgets, paramiko, tqdm.

%pip install --quiet --upgrade pip oci ipywidgets paramiko tqdm

print("Environment ready.")

In [ ]:
# Cell 2 — Objective: Define central variables; load credentials from ~/.oci/config or env; validate basics; no network calls.

import os
from datetime import datetime, timezone
import oci

# Recognized environment overrides (optional):
# - OCI_CONFIG_FILE, OCI_PROFILE
# - TENANCY_OCID, USER_OCID, FINGERPRINT
# - OCI_PRIVATE_KEY_PATH, OCI_PASSPHRASE (alias: OCI_PRIVATE_KEY_PASSPHRASE)
# - REGION
# - SSH_PUBLIC_KEY_PATH, SSH_PRIVATE_KEY_PATH
# - OUTPUT_DIR, FIO_* params

OCI_CONFIG_FILE = os.path.expanduser(os.environ.get("OCI_CONFIG_FILE", "~/.oci/config"))
OCI_PROFILE = os.environ.get("OCI_PROFILE", "DEFAULT")

_cfg = oci.config.from_file(file_location=OCI_CONFIG_FILE, profile_name=OCI_PROFILE)
oci.config.validate_config(_cfg)

TENANCY_OCID = os.environ.get("TENANCY_OCID", _cfg["tenancy"])
USER_OCID = os.environ.get("USER_OCID", _cfg["user"])
FINGERPRINT = os.environ.get("FINGERPRINT", _cfg["fingerprint"])

OCI_PRIVATE_KEY_PATH = os.path.expanduser(
    os.environ.get("OCI_PRIVATE_KEY_PATH", _cfg.get("key_file", ""))
)
PRIVATE_KEY_PASSPHRASE = os.environ.get(
    "OCI_PASSPHRASE",
    os.environ.get("OCI_PRIVATE_KEY_PASSPHRASE", _cfg.get("pass_phrase", "")),
)

REGION = os.environ.get("REGION", _cfg["region"])

# SSH keys (may be updated in Cell 3)
_default_ssh_pub = os.path.expanduser(
    os.environ.get("SSH_PUBLIC_KEY_PATH", "~/.ssh/id_rsa.pub")
)
_default_ssh_priv = os.path.expanduser(
    os.environ.get("SSH_PRIVATE_KEY_PATH", "~/.ssh/id_rsa")
)
SSH_PUBLIC_KEY_PATH = _default_ssh_pub if os.path.exists(_default_ssh_pub) else ""
SSH_PRIVATE_KEY_PATH = _default_ssh_priv if os.path.exists(_default_ssh_priv) else ""

# Test parameters (sample small matrix; adjust as desired)
SIZES = range(100, 200, 50)  # 100, 150
VPUS = range(30, 50, 10)  # 30, 40
FIO_FILE_SIZE = os.environ.get("FIO_FILE_SIZE", "10G")
FIO_BLOCK_SIZE = os.environ.get("FIO_BLOCK_SIZE", "1M")
FIO_RUNTIME = int(os.environ.get("FIO_RUNTIME", "60"))
FIO_IODEPTH = int(os.environ.get("FIO_IODEPTH", "32"))
FIO_NUMJOBS = int(os.environ.get("FIO_NUMJOBS", "1"))
RESULTS_FILE = os.environ.get("RESULTS_FILE", "fio_results.csv")

# Output directory and UTC timestamp (timezone-aware)
OUTPUT_DIR = os.environ.get("OUTPUT_DIR", ".")
os.makedirs(OUTPUT_DIR, exist_ok=True)
TS_UTC = datetime.now(timezone.utc).strftime("%Y%m%dT%H%M%SZ")
RESULTS_PATH = os.path.join(OUTPUT_DIR, RESULTS_FILE)


def _ensure_path_exists(path: str, label: str):
    if path and not os.path.exists(os.path.expanduser(path)):
        raise FileNotFoundError(f"{label} not found: {path}")


# API key must exist for Cell 3 discovery
_ensure_path_exists(OCI_PRIVATE_KEY_PATH, "OCI private key")

# SSH keys optional here (selected in Cell 3)
if not SSH_PUBLIC_KEY_PATH:
    print(
        "Note: SSH public key not found at default path; you will select a key in Cell 3."
    )
if not SSH_PRIVATE_KEY_PATH:
    print(
        "Note: SSH private key not found at default path; you will select a key in Cell 3."
    )

# Placeholders (populated in Cell 3)
SELECTED_REGION = None
COMPARTMENT_ID = ""
COMPARTMENT_NAME = ""
AVAILABILITY_DOMAIN = ""
VM_SHAPE = None
VM_OCPUS = None
VM_MEMORY_IN_GBS = None
IMAGE_ID = ""
INITIAL_VOLUME_SIZE_GBS = 100
INITIAL_VPUS_PER_GB = 30

print(
    f"Config loaded from {OCI_CONFIG_FILE} [{OCI_PROFILE}] for tenancy: {TENANCY_OCID}"
)
print(
    "Proceed to Cell 3 to select Region/Compartment/AD/Shape/Image and pick your SSH keys."
)

In [ ]:
# Cell 3 — Objective: Interactive dashboard to pick Region, Compartment, AD, Shape(+Flex OCPUs/Memory), Image, Keys; set initial volume size/VPUs.
# Single-column, centered, responsive layout; images filtered by selected Shape to ensure compatibility.

import os
from pathlib import Path
import oci
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

# Styling
SECTION_BG = "#f8f9fb"
BORDER = "1px solid #e0e0e0"
PAD = "12px"
CONTAINER_WIDTH = "100%"
CONTAINER_MAX_W = "96%"


def set_desc_width(widget, width="140px"):
    try:
        widget.style.description_width = width
    except Exception:
        pass


def section(title_text, body_widgets):
    title = widgets.HTML(f"<b>{title_text}</b>")
    body = (
        body_widgets
        if isinstance(body_widgets, widgets.Widget)
        else widgets.VBox(body_widgets)
    )
    return widgets.VBox(
        [title, body],
        layout=widgets.Layout(
            width="100%",
            border=BORDER,
            padding=PAD,
            margin="8px 0",
            background_color=SECTION_BG,
        ),
    )


def row(*children, gap="12px"):
    return widgets.HBox(list(children), layout=widgets.Layout(gap=gap, width="100%"))


def vspace(h="6px"):
    return widgets.HTML(f"<div style='height:{h}'></div>")


# OCI helpers
def build_signer(tenancy, user, fp, key_path, passphrase, cfg):
    return oci.signer.Signer(
        tenancy=tenancy,
        user=user,
        fingerprint=fp,
        private_key_file_location=key_path,
        pass_phrase=passphrase if passphrase else None,
        private_key_content=cfg.get("key_content"),
    )


def cfg_for_region(region_name):
    c = dict(_cfg)
    c["region"] = region_name
    return c


def selected_label(dropdown):
    try:
        for lbl, val in dropdown.options:
            if val == dropdown.value:
                return lbl
    except Exception:
        pass
    return ""


def discover_files(dirs, exts=None, include_hidden=True):
    out = []
    for d in dirs:
        p = Path(os.path.expanduser(d))
        if not p.exists() or not p.is_dir():
            continue
        for f in p.iterdir():
            if not f.is_file():
                continue
            if not include_hidden and f.name.startswith("."):
                continue
            if exts is not None and not any(str(f).endswith(ext) for ext in exts):
                continue
            out.append(str(f))
    out.sort(
        key=lambda s: Path(s).stat().st_mtime if Path(s).exists() else 0, reverse=True
    )
    return out


# Initial discovery using config key
_signer_base = build_signer(
    TENANCY_OCID,
    USER_OCID,
    FINGERPRINT,
    OCI_PRIVATE_KEY_PATH,
    PRIVATE_KEY_PASSPHRASE,
    _cfg,
)
idc_base = oci.identity.IdentityClient(config=_cfg, signer=_signer_base)
subs = idc_base.list_region_subscriptions(TENANCY_OCID).data
subs = sorted(subs, key=lambda r: r.region_name)
region_options = [(r.region_name, r.region_name) for r in subs]
default_region = next(
    (r.region_name for r in subs if r.is_home_region), subs[0].region_name
)

# Widgets
full = widgets.Layout(width="100%")
region_dd = widgets.Dropdown(
    options=region_options, value=default_region, description="Region:", layout=full
)
comp_filter = widgets.Text(
    value="",
    description="Comp filter:",
    placeholder="substring (optional)",
    layout=full,
)
comp_dd = widgets.Dropdown(
    options=[("Select a region first", "")], description="Compartment:", layout=full
)
ad_dd = widgets.Dropdown(
    options=[("Select a region first", "")], description="AD:", layout=full
)
for w in (region_dd, comp_filter, comp_dd, ad_dd):
    set_desc_width(w)

shape_filter = widgets.Text(
    value="",
    description="Shape filter:",
    placeholder="e.g. E5.Flex or DenseIO",
    layout=full,
)
shape_dd = widgets.Dropdown(
    options=[("Select a region first", "")], description="Shape:", layout=full
)
ocpus_in = widgets.BoundedIntText(
    value=2,
    min=1,
    max=64,
    step=1,
    description="OCPUs:",
    layout=widgets.Layout(width="200px", min_width="160px"),
)
mem_in = widgets.BoundedIntText(
    value=32,
    min=1,
    max=2048,
    step=1,
    description="Memory(GB):",
    layout=widgets.Layout(width="220px", min_width="160px"),
)
for w in (shape_filter, shape_dd, ocpus_in, mem_in):
    set_desc_width(w)
flex_hint = widgets.HTML(
    "<i>Set OCPUs/Memory for Flex shapes; hidden for fixed shapes.</i>"
)

image_dd = widgets.Dropdown(
    options=[("Select compartment first", "")], description="Image:", layout=full
)
set_desc_width(image_dd)

vol_size = widgets.BoundedIntText(
    value=100,
    min=50,
    max=32768,
    step=10,
    description="Init Size(GB):",
    layout=widgets.Layout(width="220px", min_width="160px"),
)
vol_vpus = widgets.BoundedIntText(
    value=30,
    min=10,
    max=120,
    step=10,
    description="Init VPUs/GB:",
    layout=widgets.Layout(width="220px", min_width="160px"),
)
for w in (vol_size, vol_vpus):
    set_desc_width(w)

api_key_dirs = ["~/.oci/sessions/DEFAULT", "~/.oci", "~/.ssh"]
pem_candidates = discover_files(api_key_dirs, exts=[".pem"])
other_candidates = discover_files(api_key_dirs, exts=None)
api_key_candidates = pem_candidates + [
    p for p in other_candidates if p not in pem_candidates
]
api_key_label = (
    f"Use config key_file ({OCI_PRIVATE_KEY_PATH})"
    if OCI_PRIVATE_KEY_PATH
    else "Use config key_file (unset)"
)
api_key_options = [(api_key_label, "__CONFIG__")] + (
    [(p, p) for p in api_key_candidates] or [("No key files found", "")]
)
api_key_dd = widgets.Dropdown(
    options=api_key_options, value="__CONFIG__", description="API key:", layout=full
)
set_desc_width(api_key_dd)

ssh_dir = os.path.expanduser("~/.ssh")
ssh_pub_candidates = [p for p in discover_files([ssh_dir], exts=[".pub"])]
ssh_priv_candidates = [
    p for p in discover_files([ssh_dir], exts=None) if not p.endswith(".pub")
]
ssh_pub_default = next(
    (p for p in ssh_pub_candidates if p == SSH_PUBLIC_KEY_PATH),
    (ssh_pub_candidates[0] if ssh_pub_candidates else ""),
)
ssh_priv_default = next(
    (p for p in ssh_priv_candidates if p == SSH_PRIVATE_KEY_PATH),
    (ssh_priv_candidates[0] if ssh_priv_candidates else ""),
)

ssh_pub_dd = widgets.Dropdown(
    options=[(p, p) for p in ssh_pub_candidates]
    or [("No *.pub keys found in ~/.ssh", "")],
    value=ssh_pub_default,
    description="SSH pub:",
    layout=full,
)
set_desc_width(ssh_pub_dd)
ssh_priv_dd = widgets.Dropdown(
    options=[(p, p) for p in ssh_priv_candidates]
    or [("No private keys found in ~/.ssh", "")],
    value=ssh_priv_default,
    description="SSH priv:",
    layout=full,
)
set_desc_width(ssh_priv_dd)

apply_btn = widgets.Button(
    description="Apply Selections",
    button_style="primary",
    icon="check",
    layout=widgets.Layout(width="240px", height="36px", align_self="center"),
)
out = widgets.Output(layout=widgets.Layout(width="100%"))

# Stores
_shape_info_by_name = {}
_all_comp_options = []


# Loaders / refreshers
def current_api_key_path():
    return (
        OCI_PRIVATE_KEY_PATH
        if api_key_dd.value == "__CONFIG__"
        else os.path.expanduser(api_key_dd.value or OCI_PRIVATE_KEY_PATH)
    )


def refresh_compartments_and_ads(*_):
    global _all_comp_options
    cfg_r = cfg_for_region(region_dd.value)
    signer = build_signer(
        TENANCY_OCID,
        USER_OCID,
        FINGERPRINT,
        current_api_key_path(),
        PRIVATE_KEY_PASSPHRASE,
        cfg_r,
    )
    idc = oci.identity.IdentityClient(config=cfg_r, signer=signer)

    comps = oci.pagination.list_call_get_all_results(
        idc.list_compartments,
        TENANCY_OCID,
        compartment_id_in_subtree=True,
        access_level="ACCESSIBLE",
    ).data
    comps = [c for c in comps if c.lifecycle_state == "ACTIVE"]
    tenancy = idc.get_tenancy(TENANCY_OCID).data
    tenancy_name = getattr(tenancy, "name", "root-tenancy")

    _all_comp_options = [(f"{tenancy_name} (root)", TENANCY_OCID)] + sorted(
        [(f"{c.name} ({c.description or 'no-desc'})", c.id) for c in comps],
        key=lambda t: t[0].lower(),
    )
    comp_dd.options = _all_comp_options
    comp_dd.value = _all_comp_options[0][1] if _all_comp_options else ""

    ads = oci.pagination.list_call_get_all_results(
        idc.list_availability_domains, TENANCY_OCID
    ).data
    ad_dd.options = [(ad.name, ad.name) for ad in ads] or [("No ADs found", "")]
    ad_dd.value = ad_dd.options[0][1] if ad_dd.options else ""

    refresh_shapes()
    refresh_images()


def refresh_shapes(*_):
    cfg_r = cfg_for_region(region_dd.value)
    signer = build_signer(
        TENANCY_OCID,
        USER_OCID,
        FINGERPRINT,
        current_api_key_path(),
        PRIVATE_KEY_PASSPHRASE,
        cfg_r,
    )
    cc = oci.core.ComputeClient(config=cfg_r, signer=signer)

    shapes = oci.pagination.list_call_get_all_results(cc.list_shapes, TENANCY_OCID).data
    _shape_info_by_name.clear()
    for s in shapes:
        if s.shape not in _shape_info_by_name:
            _shape_info_by_name[s.shape] = s

    names = sorted(_shape_info_by_name.keys())
    if shape_filter.value:
        names = [n for n in names if shape_filter.value.lower() in n.lower()]
    opts = [(n, n) for n in names] or [("No shapes match filter", "")]
    shape_dd.options = opts
    shape_dd.value = opts[0][1] if opts else ""
    show_hide_flex_inputs()
    refresh_images()


def show_hide_flex_inputs(*_):
    is_flex = (shape_dd.value or "").endswith(".Flex")
    ocpus_in.layout.display = "block" if is_flex else "none"
    mem_in.layout.display = "block" if is_flex else "none"
    if is_flex:
        s_info = _shape_info_by_name.get(shape_dd.value)
        max_ocpus = int(getattr(getattr(s_info, "ocpu_options", None), "max", 64) or 64)
        ocpus_in.max = max_ocpus
        if getattr(s_info, "memory_options", None):
            mem_in.max = int(s_info.memory_options.max_in_g_bs)
        else:
            mem_in.max = 2048


def on_comp_filter_change(*_):
    if not _all_comp_options:
        return
    text = comp_filter.value.strip().lower()
    comp_dd.options = (
        _all_comp_options[:]
        if not text
        else [p for p in _all_comp_options if text in p[0].lower()]
    )


def refresh_images(*_):
    cfg_r = cfg_for_region(region_dd.value)
    signer = build_signer(
        TENANCY_OCID,
        USER_OCID,
        FINGERPRINT,
        current_api_key_path(),
        PRIVATE_KEY_PASSPHRASE,
        cfg_r,
    )
    cc = oci.core.ComputeClient(config=cfg_r, signer=signer)

    comp_id = comp_dd.value
    if not comp_id:
        image_dd.options = [("Select compartment first", "")]
        image_dd.value = ""
        return

    shape_name = (
        shape_dd.value
        if shape_dd.value and shape_dd.value != "Select a region first"
        else None
    )
    if shape_name:
        imgs = oci.pagination.list_call_get_all_results(
            cc.list_images,
            comp_id,
            shape=shape_name,
            operating_system="Oracle Linux",
            sort_by="TIMECREATED",
            sort_order="DESC",
        ).data
    else:
        imgs = oci.pagination.list_call_get_all_results(
            cc.list_images,
            comp_id,
            operating_system="Oracle Linux",
            sort_by="TIMECREATED",
            sort_order="DESC",
        ).data

    items = []
    for im in imgs[:100]:
        try:
            created = im.time_created.strftime("%Y-%m-%d")
        except Exception:
            created = ""
        items.append(
            (
                f"{im.display_name} — {im.operating_system} {im.operating_system_version} — {created}",
                im.id,
            )
        )

    if not items:
        image_dd.options = [
            (
                "No compatible Oracle Linux images found for this shape; try another shape or compartment",
                "",
            )
        ]
        image_dd.value = ""
        return

    image_dd.options = items
    image_dd.value = items[0][1]


def render_summary():
    comp_lbl = selected_label(comp_dd)
    oc = (
        f"OCPUs: {VM_OCPUS}, Memory(GB): {VM_MEMORY_IN_GBS}"
        if VM_OCPUS is not None
        else "OCPUs/Memory: (fixed shape)"
    )
    html = f"""
    <div style="border:{BORDER};background:{SECTION_BG};padding:{PAD};width:100%;">
      <b>Selections applied</b>
      <div style="font-family:ui-monospace, SFMono-Regular, Menlo, Monaco, Consolas, 'Liberation Mono', 'Courier New', monospace; white-space:pre-wrap; word-break:break-word;">
Region: {REGION}
Compartment: {comp_lbl} -> {COMPARTMENT_ID}
AD: {AVAILABILITY_DOMAIN}
Shape: {VM_SHAPE}
{oc}
Image: {IMAGE_ID}
Initial Volume: size={INITIAL_VOLUME_SIZE_GBS} GB, vpus/gb={INITIAL_VPUS_PER_GB}
API key path: {"(from config)" if api_key_dd.value=="__CONFIG__" else OCI_PRIVATE_KEY_PATH}
SSH pub: {SSH_PUBLIC_KEY_PATH}
SSH priv: {SSH_PRIVATE_KEY_PATH}
      </div>
    </div>
    """
    return HTML(html)


def on_apply_clicked(_):
    global SELECTED_REGION, REGION, COMPARTMENT_ID, COMPARTMENT_NAME, AVAILABILITY_DOMAIN
    global VM_SHAPE, VM_OCPUS, VM_MEMORY_IN_GBS, IMAGE_ID, INITIAL_VOLUME_SIZE_GBS, INITIAL_VPUS_PER_GB
    global OCI_PRIVATE_KEY_PATH, SSH_PUBLIC_KEY_PATH, SSH_PRIVATE_KEY_PATH

    SELECTED_REGION = region_dd.value
    REGION = SELECTED_REGION
    COMPARTMENT_ID = comp_dd.value
    COMPARTMENT_NAME = selected_label(comp_dd)
    AVAILABILITY_DOMAIN = ad_dd.value

    VM_SHAPE = shape_dd.value
    if (VM_SHAPE or "").endswith(".Flex"):
        VM_OCPUS = float(ocpus_in.value)
        VM_MEMORY_IN_GBS = float(mem_in.value)
    else:
        VM_OCPUS = None
        VM_MEMORY_IN_GBS = None

    IMAGE_ID = image_dd.value
    INITIAL_VOLUME_SIZE_GBS = int(vol_size.value)
    INITIAL_VPUS_PER_GB = int(vol_vpus.value)

    if api_key_dd.value and api_key_dd.value != "__CONFIG__":
        OCI_PRIVATE_KEY_PATH = os.path.expanduser(api_key_dd.value)
    SSH_PUBLIC_KEY_PATH = os.path.expanduser(ssh_pub_dd.value or SSH_PUBLIC_KEY_PATH)
    SSH_PRIVATE_KEY_PATH = os.path.expanduser(ssh_priv_dd.value or SSH_PRIVATE_KEY_PATH)

    # Export env for Terraform + SDK
    os.environ["OCI_CONFIG_FILE"] = OCI_CONFIG_FILE
    os.environ["OCI_PROFILE"] = OCI_PROFILE
    os.environ["REGION"] = REGION
    os.environ["COMPARTMENT_ID"] = COMPARTMENT_ID
    os.environ["AVAILABILITY_DOMAIN"] = AVAILABILITY_DOMAIN
    os.environ["IMAGE_ID"] = IMAGE_ID

    with out:
        out.clear_output()
        display(render_summary())


# Wire events
region_dd.observe(refresh_compartments_and_ads, names="value")
shape_filter.observe(refresh_shapes, names="value")
comp_filter.observe(on_comp_filter_change, names="value")
shape_dd.observe(show_hide_flex_inputs, names="value")
shape_dd.observe(refresh_images, names="value")
comp_dd.observe(refresh_images, names="value")
apply_btn.on_click(on_apply_clicked)

# Initial population + UI
refresh_compartments_and_ads()
container = widgets.VBox(
    [
        section(
            "Location",
            [
                row(region_dd),
                vspace(),
                row(comp_filter),
                vspace(),
                row(comp_dd),
                vspace(),
                row(ad_dd),
            ],
        ),
        section(
            "Shape and Flex configuration",
            [
                row(shape_filter),
                vspace(),
                row(shape_dd),
                vspace(),
                row(ocpus_in, mem_in),
                vspace("2px"),
                flex_hint,
            ],
        ),
        section(
            "Image (auto: newest Oracle Linux in selected compartment; filtered by Shape)",
            row(image_dd),
        ),
        section("Initial Block Volume", row(vol_size, vol_vpus)),
        section(
            "Keys (API + SSH)",
            [row(api_key_dd), vspace(), row(ssh_pub_dd), vspace(), row(ssh_priv_dd)],
        ),
        widgets.HBox(
            [apply_btn], layout=widgets.Layout(justify_content="center", width="100%")
        ),
        out,
    ],
    layout=widgets.Layout(
        width=CONTAINER_WIDTH, max_width=CONTAINER_MAX_W, margin="0 auto"
    ),
)
display(container)

In [ ]:
# Cell 4 — Objective: Write Terraform Configuration (provider via config_file_profile and region; variables for volume/shape).

import os

shape_config_block = ""
if (VM_SHAPE or "").endswith(".Flex"):
    shape_config_block = f"""
  shape_config {{
    ocpus         = {VM_OCPUS}
    memory_in_gbs = {VM_MEMORY_IN_GBS}
  }}"""

terraform_config = f"""
terraform {{
  required_providers {{
    oci = {{
      source  = "oracle/oci"
      version = ">= 5.0.0"
    }}
  }}
}}

provider "oci" {{
  config_file_profile = var.oci_profile
  region              = var.region
}}

variable "oci_profile" {{}}
variable "region" {{}}
variable "compartment_id" {{}}
variable "availability_domain" {{}}
variable "image_id" {{}}
variable "ssh_public_key_content" {{}}
variable "ssh_private_key_path" {{}}
variable "vm_shape" {{}}
variable "initial_volume_size_gbs" {{}}
variable "initial_vpus_per_gb" {{}}

resource "oci_core_vcn" "test_vcn" {{
  cidr_block     = "10.0.0.0/16"
  compartment_id = var.compartment_id
  display_name   = "TestVCN"
}}

resource "oci_core_internet_gateway" "test_igw" {{
  compartment_id = var.compartment_id
  vcn_id         = oci_core_vcn.test_vcn.id
  display_name   = "TestInternetGateway"
}}

resource "oci_core_route_table" "test_route_table" {{
  compartment_id = var.compartment_id
  vcn_id         = oci_core_vcn.test_vcn.id
  display_name   = "TestRouteTable"
  route_rules {{
    destination       = "0.0.0.0/0"
    destination_type  = "CIDR_BLOCK"
    network_entity_id = oci_core_internet_gateway.test_igw.id
  }}
}}

resource "oci_core_security_list" "test_security_list" {{
  compartment_id = var.compartment_id
  vcn_id         = oci_core_vcn.test_vcn.id
  display_name   = "TestSecurityList"
  ingress_security_rules {{
    protocol    = "6"
    source      = "0.0.0.0/0"
    source_type = "CIDR_BLOCK"
    tcp_options {{
      min = 22
      max = 22
    }}
  }}
  egress_security_rules {{
    protocol    = "all"
    destination = "0.0.0.0/0"
  }}
}}

resource "oci_core_subnet" "test_subnet" {{
  cidr_block        = "10.0.1.0/24"
  compartment_id    = var.compartment_id
  vcn_id            = oci_core_vcn.test_vcn.id
  display_name      = "TestSubnet"
  route_table_id    = oci_core_route_table.test_route_table.id
  security_list_ids = [oci_core_security_list.test_security_list.id]
}}

resource "oci_core_instance" "test_instance" {{
  availability_domain = var.availability_domain
  compartment_id      = var.compartment_id
  shape               = var.vm_shape{shape_config_block}
  source_details {{
    source_type = "image"
    source_id   = var.image_id
  }}
  create_vnic_details {{
    subnet_id        = oci_core_subnet.test_subnet.id
    assign_public_ip = true
  }}
  display_name = "TestVM"
  metadata = {{
    ssh_authorized_keys = var.ssh_public_key_content
  }}

  provisioner "remote-exec" {{
    inline = [
      "set -e",
      "if command -v dnf >/dev/null 2>&1; then sudo dnf -y install fio xfsprogs",
      "elif command -v microdnf >/dev/null 2>&1; then sudo microdnf -y install fio xfsprogs",
      "elif command -v yum >/dev/null 2>&1; then sudo yum -y install fio xfsprogs",
      "elif command -v apt-get >/dev/null 2>&1; then sudo bash -lc 'DEBIAN_FRONTEND=noninteractive apt-get update -y && apt-get install -y fio xfsprogs'",
      "else echo 'No supported package manager found (dnf/microdnf/yum/apt-get). Continuing without preinstall.'; fi",
      "sudo mkdir -p /export",
      "sudo chown opc:opc /export"
    ]
    connection {{
      type        = "ssh"
      user        = "opc"
      private_key = file(var.ssh_private_key_path)
      host        = self.public_ip
      timeout     = "5m"
    }}
  }}
}}

resource "oci_core_volume" "test_volume" {{
  compartment_id      = var.compartment_id
  availability_domain = var.availability_domain
  display_name        = "TestBlockVolume"
  size_in_gbs         = var.initial_volume_size_gbs
  vpus_per_gb         = var.initial_vpus_per_gb
}}

resource "oci_core_volume_attachment" "test_volume_attachment" {{
  attachment_type = "paravirtualized"
  instance_id     = oci_core_instance.test_instance.id
  volume_id       = oci_core_volume.test_volume.id
}}

output "instance_id" {{ value = oci_core_instance.test_instance.id }}
output "volume_id"    {{ value = oci_core_volume.test_volume.id }}
output "vm_public_ip" {{ value = oci_core_instance.test_instance.public_ip }}
"""

with open("main.tf", "w") as f:
    f.write(terraform_config)
print("Terraform configuration saved as main.tf")

In [ ]:
# Cell 5 — Objective: Write terraform.tfvars using dashboard selections and paths only (no secrets).

import os

if not IMAGE_ID:
    raise ValueError("IMAGE_ID is not set. Run Cell 3 and select an Image.")
if not VM_SHAPE:
    raise ValueError("VM_SHAPE is not set. Run Cell 3 and select a Shape.")
if not SSH_PUBLIC_KEY_PATH or not os.path.exists(SSH_PUBLIC_KEY_PATH):
    raise ValueError(
        f"SSH public key file is missing or not found. Pick a valid key in Cell 3. Current: {SSH_PUBLIC_KEY_PATH}"
    )
if not SSH_PRIVATE_KEY_PATH or not os.path.exists(SSH_PRIVATE_KEY_PATH):
    raise ValueError(
        f"SSH private key file is missing or not found. Pick a valid key in Cell 3. Current: {SSH_PRIVATE_KEY_PATH}"
    )

with open(os.path.expanduser(SSH_PUBLIC_KEY_PATH), "r") as f:
    ssh_public_key_content = f.read().strip()

terraform_vars = f"""
oci_profile             = "{OCI_PROFILE}"
region                  = "{REGION}"
compartment_id          = "{COMPARTMENT_ID}"
availability_domain     = "{AVAILABILITY_DOMAIN}"
image_id                = "{IMAGE_ID}"
ssh_public_key_content  = "{ssh_public_key_content}"
ssh_private_key_path    = "{os.path.expanduser(SSH_PRIVATE_KEY_PATH)}"
vm_shape                = "{VM_SHAPE}"
initial_volume_size_gbs = {int(INITIAL_VOLUME_SIZE_GBS)}
initial_vpus_per_gb     = {int(INITIAL_VPUS_PER_GB)}
""".lstrip()

with open("terraform.tfvars", "w") as f:
    f.write(terraform_vars)
print("Terraform variables saved as terraform.tfvars")

In [ ]:
# Cell 6 — Objective: Initialize and apply Terraform to create resources.

!terraform init
!terraform apply -auto-approve -var-file=terraform.tfvars


In [ ]:
# Cell 7 — Objective: Extract instance_id, volume_id, vm_public_ip from Terraform outputs.

import json, subprocess


def get_terraform_outputs():
    result = subprocess.run(
        ["terraform", "output", "-json"], capture_output=True, text=True
    )
    if result.returncode == 0:
        outputs = json.loads(result.stdout or "{}")
        return (
            outputs.get("instance_id", {}).get("value"),
            outputs.get("volume_id", {}).get("value"),
            outputs.get("vm_public_ip", {}).get("value"),
        )
    else:
        print("Error getting Terraform outputs:", result.stderr)
        return None, None, None


instance_id, volume_id, vm_public_ip = get_terraform_outputs()
print(f"Instance ID: {instance_id}")
print(f"Volume ID: {volume_id}")
print(f"VM Public IP: {vm_public_ip}")

In [ ]:
# Cell 8 — Objective: Update block volume size/VPUs across matrix; prepare filesystem (format once, then xfs_growfs), run FIO; export CSV (with metadata).

import oci
import paramiko
import time
import csv
import re
from tqdm import tqdm

# Guards
if not vm_public_ip:
    raise RuntimeError(
        "vm_public_ip is empty. Run Cells 6 and 7 to provision and fetch Terraform outputs."
    )
if not SSH_PRIVATE_KEY_PATH:
    raise RuntimeError(
        "SSH_PRIVATE_KEY_PATH is empty. Select keys in Cell 3 and run Cell 5 again."
    )
if "volume_id" not in globals() or not volume_id:
    raise RuntimeError("volume_id is empty. Run Cell 7 to fetch Terraform outputs.")

# OCI SDK Clients (respect selections from Cell 2/3)
config = oci.config.from_file(OCI_CONFIG_FILE, OCI_PROFILE)
config["region"] = REGION  # ensure active region matches selections
block_storage_client = oci.core.BlockstorageClient(config)

# VM and SSH settings
ssh_host = vm_public_ip
ssh_user = "opc"
ssh_key = SSH_PRIVATE_KEY_PATH

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())


def run_ssh_command(command, timeout=120, retries=5, delay=10):
    """Run a command over SSH with retries and return (stdout, exit_status)."""
    for attempt in range(retries):
        try:
            ssh.connect(
                ssh_host, username=ssh_user, key_filename=ssh_key, timeout=timeout
            )
            stdin, stdout, stderr = ssh.exec_command(command, timeout=timeout)
            output = stdout.read().decode("utf-8").strip()
            error = stderr.read().decode("utf-8").strip()
            exit_status = stdout.channel.recv_exit_status()
            ssh.close()
            if error:
                print(f"SSH stderr for '{command}': {error}")
            return output, exit_status
        except Exception as e:
            print(
                f"SSH attempt {attempt+1}/{retries} failed for '{command}': {e}; retrying in {delay}s"
            )
            time.sleep(delay)
    return None, 1


def ensure_tools_installed():
    """
    Ensure fio and xfs_growfs (xfsprogs) are installed on the instance.
    Supports dnf, microdnf, yum, apt-get.
    """
    # Check fio
    out, _ = run_ssh_command(
        'sudo bash -lc "command -v fio >/dev/null 2>&1 && echo OK || echo MISS"'
    )
    fio_ok = out == "OK"
    # Check xfs_growfs
    out, _ = run_ssh_command(
        'sudo bash -lc "command -v xfs_growfs >/dev/null 2>&1 && echo OK || echo MISS"'
    )
    xfs_ok = out == "OK"

    if fio_ok and xfs_ok:
        return True

    print("Installing missing tools (fio/xfsprogs) via available package manager...")
    pm_sequence = [
        (
            "dnf",
            'sudo bash -lc "command -v dnf >/dev/null 2>&1 && echo YES || echo NO"',
            "sudo dnf -y install fio xfsprogs",
        ),
        (
            "microdnf",
            'sudo bash -lc "command -v microdnf >/dev/null 2>&1 && echo YES || echo NO"',
            "sudo microdnf -y install fio xfsprogs",
        ),
        (
            "yum",
            'sudo bash -lc "command -v yum >/dev/null 2>&1 && echo YES || echo NO"',
            "sudo yum -y install fio xfsprogs",
        ),
        (
            "apt-get",
            'sudo bash -lc "command -v apt-get >/dev/null 2>&1 && echo YES || echo NO"',
            'sudo bash -lc "DEBIAN_FRONTEND=noninteractive apt-get update -y && apt-get install -y fio xfsprogs"',
        ),
    ]
    installed = False
    for pm, check_cmd, install_cmd in pm_sequence:
        out, _ = run_ssh_command(check_cmd)
        if out == "YES":
            print(f"Installing via {pm} ...")
            _, status = run_ssh_command(install_cmd, timeout=900, retries=3, delay=20)
            installed = status == 0
            break

    # Re-check
    out, _ = run_ssh_command(
        'sudo bash -lc "command -v fio >/dev/null 2>&1 && echo OK || echo MISS"'
    )
    fio_ok2 = out == "OK"
    out, _ = run_ssh_command(
        'sudo bash -lc "command -v xfs_growfs >/dev/null 2>&1 && echo OK || echo MISS"'
    )
    xfs_ok2 = out == "OK"

    if fio_ok2 and xfs_ok2:
        print("fio and xfs_growfs are available.")
        return True

    print("Failed to install fio/xfsprogs or not in sudo PATH.")
    return False


def update_block_volume(size_in_gbs, vpus_per_gb, retries=3, delay=60):
    """Update the OCI Block Volume size and VPUs; wait until AVAILABLE."""
    for attempt in range(retries):
        try:
            update_details = oci.core.models.UpdateVolumeDetails(
                size_in_gbs=size_in_gbs, vpus_per_gb=vpus_per_gb
            )
            block_storage_client.update_volume(volume_id, update_details)
            print(f"Requested update: size={size_in_gbs} GB, vpus/gb={vpus_per_gb}")
            oci.wait_until(
                block_storage_client,
                block_storage_client.get_volume(volume_id),
                "lifecycle_state",
                "AVAILABLE",
                max_wait_seconds=600,
            )
            print("Block Volume updated and AVAILABLE.")
            return True
        except oci.exceptions.ServiceError as e:
            if e.status == 429:
                print(
                    f"OCI rate limit (429). Attempt {attempt+1}/{retries}; retrying in {delay}s..."
                )
                time.sleep(delay)
            else:
                print(f"OCI ServiceError: {e}")
                return False
        except Exception as e:
            print(f"Unexpected error updating volume: {e}")
            return False
    return False


def detect_device_by_size(size_in_gbs):
    """Detect the block device (~expected size) to use (not sda)."""
    expected_size_bytes = size_in_gbs * 1024 * 1024 * 1024
    detect_cmd = (
        f"lsblk -bno NAME,SIZE,TYPE | "
        f"awk '$2 > {expected_size_bytes - 10*1024*1024*1024} && "
        f"$2 < {expected_size_bytes + 10*1024*1024*1024} && "
        f'$3 == "disk" && $1 != "sda" {{print $1}}\' | head -1'
    )
    device = run_ssh_command(detect_cmd)[0]
    if not device:
        print("No device match; triggering global SCSI rescan...")
        run_ssh_command(
            "sudo sh -c 'for d in /sys/class/scsi_device/*; do echo 1 > $d/device/rescan; done'"
        )
        time.sleep(10)
        device = run_ssh_command(detect_cmd)[0]
    return device


def prepare_filesystem(size_in_gbs):
    """
    Prepare /export with XFS:
    - First time: mkfs.xfs + mount.
    - Subsequent runs (after volume grows): xfs_growfs /export (no reformat).
    """
    device = detect_device_by_size(size_in_gbs)
    if not device:
        print("Could not detect device after rescan.")
        return False
    print(f"Detected candidate device: /dev/{device}")

    # Is /export already mounted?
    mount_out, mount_rc = run_ssh_command("mount | grep ' /export '")
    if mount_rc != 0:
        # Not mounted yet — format and mount the first time
        cmds = [
            f"sudo sh -lc 'mkfs.xfs -f /dev/{device}'",
            f"sudo sh -lc 'mkdir -p /export && mount /dev/{device} /export'",
            "sudo chown opc:opc /export",
        ]
        for cmd in cmds:
            _, rc = run_ssh_command(cmd)
            if rc != 0:
                print("Formatting/mounting failed.")
                return False
            time.sleep(2)
        print("Formatted and mounted /export.")
        return True
    else:
        # Already mounted — grow filesystem online
        grow_out, grow_rc = run_ssh_command("sudo sh -lc 'xfs_growfs /export'")
        if grow_rc == 0:
            print("xfs_growfs completed.")
            return True
        # Fallback: remount and format if growth failed
        print("xfs_growfs failed; attempting reformat as fallback.")
        run_ssh_command("sudo umount /export || true")
        cmds = [
            f"sudo sh -lc 'mkfs.xfs -f /dev/{device}'",
            f"sudo sh -lc 'mount /dev/{device} /export'",
            "sudo chown opc:opc /export",
        ]
        for cmd in cmds:
            _, rc = run_ssh_command(cmd)
            if rc != 0:
                print("Fallback reformat/mount failed.")
                return False
            time.sleep(2)
        print("Reformatted and mounted /export (fallback).")
        return True


def run_fio_tests(size, vpus, retries=3, delay=20):
    """Run sequential write/read FIO tests and parse bandwidth (MiB/s and MB/s)."""
    if not ensure_tools_installed():
        print("Cannot run FIO: installation failed.")
        return float("nan"), float("nan"), float("nan"), float("nan")

    # Prepare export dir
    pre_cmd = "sudo bash -lc 'rm -f /export/test_file /export/fio_*.log; touch /export/test_write && rm /export/test_write && echo OK'"
    output, status = run_ssh_command(pre_cmd)
    if status != 0 or output != "OK":
        print("Pre-FIO check failed.")
        return float("nan"), float("nan"), float("nan"), float("nan")

    # Write test
    write_cmd = (
        f'sudo bash -lc "fio --name=seqwrite --filename=/export/test_file --size={FIO_FILE_SIZE} --bs={FIO_BLOCK_SIZE} '
        f"--rw=write --ioengine=libaio --iodepth={FIO_IODEPTH} --direct=1 --numjobs={FIO_NUMJOBS} "
        f'--runtime={FIO_RUNTIME} --time_based --group_reporting --output=/export/fio_write.log"'
    )
    _, write_status = run_ssh_command(
        write_cmd, timeout=900, retries=retries, delay=delay
    )
    if write_status != 0:
        wl = run_ssh_command("sudo bash -lc 'cat /export/fio_write.log || true'")[0]
        print(f"FIO write failed (exit {write_status}). Log:\n{wl}")
        return float("nan"), float("nan"), float("nan"), float("nan")

    wlog = run_ssh_command("sudo bash -lc 'cat /export/fio_write.log'")[0]
    w_match = re.search(r"WRITE:\s+bw=(\d+\.?\d*)MiB/s\s+\((\d+\.?\d*)MB/s\)", wlog)
    write_mib = float(w_match.group(1)) if w_match else float("nan")
    write_mb = float(w_match.group(2)) if w_match else float("nan")

    # Read test
    read_cmd = (
        f'sudo bash -lc "fio --name=seqread --filename=/export/test_file --size={FIO_FILE_SIZE} --bs={FIO_BLOCK_SIZE} '
        f"--rw=read --ioengine=libaio --iodepth={FIO_IODEPTH} --direct=1 --numjobs={FIO_NUMJOBS} "
        f'--runtime={FIO_RUNTIME} --time_based --group_reporting --output=/export/fio_read.log"'
    )
    _, read_status = run_ssh_command(
        read_cmd, timeout=900, retries=retries, delay=delay
    )
    if read_status != 0:
        rl = run_ssh_command("sudo bash -lc 'cat /export/fio_read.log || true'")[0]
        print(f"FIO read failed (exit {read_status}). Log:\n{rl}")
        return write_mib, write_mb, float("nan"), float("nan")

    rlog = run_ssh_command("sudo bash -lc 'cat /export/fio_read.log'")[0]
    r_match = re.search(r"READ:\s+bw=(\d+\.?\d*)MiB/s\s+\((\d+\.?\d*)MB/s\)", rlog)
    read_mib = float(r_match.group(1)) if r_match else float("nan")
    read_mb = float(r_match.group(2)) if r_match else float("nan")

    return write_mib, read_mib, write_mb, read_mb


def run_tests():
    """
    Main test matrix over SIZES × VPUS; writes CSV results to RESULTS_PATH
    with metadata columns:
      Region, Shape, OCPUs, Memory(GB), Block Size, IO Depth, Runtime(s), NumJobs
    """
    results_file_path = RESULTS_PATH

    # Resolve metadata values (shape/ocpus/memory may be None for fixed shapes)
    shape_val = VM_SHAPE
    ocpus_val = VM_OCPUS if VM_OCPUS is not None else "-"
    mem_val = VM_MEMORY_IN_GBS if VM_MEMORY_IN_GBS is not None else "-"
    bs_val = FIO_BLOCK_SIZE
    iodepth_val = FIO_IODEPTH
    runtime_s = FIO_RUNTIME
    numjobs = FIO_NUMJOBS
    region_val = REGION

    with open(results_file_path, mode="w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(
            [
                "Region",
                "Shape",
                "OCPUs",
                "Memory(GB)",
                "Block Size",
                "IO Depth",
                "Runtime(s)",
                "NumJobs",
                "Size (GB)",
                "VPUs/GB",
                "Write (MiB/s)",
                "Read (MiB/s)",
                "Write (MB/s)",
                "Read (MB/s)",
            ]
        )

        total = len(list(SIZES)) * len(list(VPUS))
        with tqdm(total=total, desc="Running FIO Tests") as pbar:
            for size in SIZES:
                for vpu in VPUS:
                    print(f"\n=== Test: size={size} GB, vpus/gb={vpu} ===")
                    if update_block_volume(size, vpu):
                        if prepare_filesystem(size):
                            time.sleep(5)  # short settle
                            w_mib, r_mib, w_mb, r_mb = run_fio_tests(size, vpu)
                            writer.writerow(
                                [
                                    region_val,
                                    shape_val,
                                    ocpus_val,
                                    mem_val,
                                    bs_val,
                                    iodepth_val,
                                    runtime_s,
                                    numjobs,
                                    size,
                                    vpu,
                                    w_mib,
                                    r_mib,
                                    w_mb,
                                    r_mb,
                                ]
                            )
                            print(f"Completed test: size={size} GB, vpus/gb={vpu}")
                        else:
                            writer.writerow(
                                [
                                    region_val,
                                    shape_val,
                                    ocpus_val,
                                    mem_val,
                                    bs_val,
                                    iodepth_val,
                                    runtime_s,
                                    numjobs,
                                    size,
                                    vpu,
                                    "FS-Prep-Failed",
                                    "FS-Prep-Failed",
                                    "FS-Prep-Failed",
                                    "FS-Prep-Failed",
                                ]
                            )
                            print("Filesystem prep failed; recorded error in CSV.")
                    else:
                        writer.writerow(
                            [
                                region_val,
                                shape_val,
                                ocpus_val,
                                mem_val,
                                bs_val,
                                iodepth_val,
                                runtime_s,
                                numjobs,
                                size,
                                vpu,
                                "Vol-Update-Failed",
                                "Vol-Update-Failed",
                                "Vol-Update-Failed",
                                "Vol-Update-Failed",
                            ]
                        )
                        print("Volume update failed; recorded error in CSV.")
                    pbar.update(1)

    print(f"\nResults written to: {results_file_path}")


# Execute test matrix
run_tests()

In [ ]:
# Cell 9 — Objective: Destroy all resources created by Terraform.
!terraform destroy -auto-approve -var-file=terraform.tfvars